## 生成モデル
- データとして観測される観測変数が何らかの確率モデルから生成されていると仮定し，その生成過程を確率分布によってモデル化するアプローチ
    - 観測変数の背景にある因子(確率変数)として潜在変数も仮定することが多い
- 「データがどのようにできているのか？」を明示的に表すことができ，モデルからデータを生成することができる

## 深層生成モデル
- 観測変数が複雑な場合，単純な確率分布では直接表現できない
    - 特に観測変数がベクトルでその要素(次元)間の依存関係が非線形な場合(画像など)
        - 非線形な関係性を表すには? -> 深層ニューラルネットワーク(DNN)
            - 非線形な関係性とは?
- 深層生成モデル(deep generatice mdoels)
    - 確率分布をDNNで表現した生成モデル
    - DNNによって複雑な入力をend-to-endに扱えるようになった
- 生成モデルによって明示的に生成過程をモデル化できる + DNNによって非線形な関係性を捉えられる

## 従来の複雑な深層生成モデルを実装することが困難
- どういう点でだろうか？M2-model, TD-VAE, FactorVAE

## 深層生成モデルの特徴
- 生成モデルを構成するDNNは確率分布によって隠蔽される
    - 隠蔽されるとは？
- モデルの種類や確率変数の正則化は目的関数(誤差関数)として記述される
- モデルの学習方法はモデル(目的関数)に依存しない

### DNNの隠蔽
- 深層生成モデルを構成する確率分布はDNNによって表現される
    - DNNの構造の詳細は確率分布によって隠蔽される
        - 近年の複雑な深層生成モデルの論文では生成モデルの説明部分ではDNNの詳細は触れられない
            - TD-VAEの論文の中では，各分布を構成するDNNの詳細はAppendixに回されている
- 既存の確率プログラミング言語ではDNNと確率分布を混ぜて書く枠組み(隠蔽ができない)
    - DNNの構造を気にせずに確率分布の操作によって生成モデルを実装できる仕組みが望ましい

#### DNNによる確率分布の表現方法の違い
- DNNで確率分布を表す方法は深層生成モデルの種類によって異なる
    - 条件付き分布p(x|z)をモデル化する
        - VAE, GANなど
    - p(x)を直接モデル化する
        - 自己回帰モデル: 観測変数の各要素の条件付き分布の積で表現
        - flowベースモデル: flowによる変数変換として表現
- 様々な深層生成モデルを統一的に扱うためにも確率分布の操作とそれを構成するDNNの定義は分離する必要がある

### 目的関数によるモデルの定義
- 深層生成モデルではいずれのモデルも最適化するための目的関数を明示的に設定する
    - 自己回帰モデル・フローベースモデル: Kullback-Leiblerダイバージェンス(対数尤度)
    - VAE: 周辺対数尤度の下界
    - GAN: Jensen-Shannonダイバージェンス(ただし目的関数自身の更新も必要)
    
- 推論, 確率変数の表現の正則化なども全て目的関数として追加する
    - 深層生成モデルではモデルの設計=目的関数の定義
    - 従来の生成モデルと異なりサンプリングによる推論等は行わない
        - これを実現するには確率分布を受け取って目的関数を定義できる枠組みが必要

### 学習方法はモデルに依存しない
- 深層生成モデルではモデルに依存せずに勾配降下法で学習する
    - 通常のDNNの学習と同様，様々なモデルに対して任意の最適化アルゴリズムを用いる
    - 従来の生成モデルのようにモデルに応じて決まった学習アルゴリズムが選択されることはない
        - ギブスサンプリング，EMアルゴリズム，変分推論
- 目的関数と最適化アルゴリズムが独立に設定・学習できる枠組みが必要

## 深層生成モデルの特徴を考慮したAPI
- 生成モデルを構成するDNNは確率分布によって隠蔽される
    - DNNの定義と確率分布の操作を分離できる枠組み(Distribution API)
- モデルの種類や確率変数の正則化は目的関数(誤差関数)として記述される
    - 確率分布を受け取って目的関数を定義できる枠組み(Loss API)
- モデルの学習方法はモデル(目的関数)に依存しない
    - 目的関数と最適化アルゴリズムが独立に設定できる枠組み(Model API)

In [18]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms

from tqdm import tqdm

batch_size = 128
epochs = 10
seed = 1
torch.manual_seed(seed)

root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1))])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

In [19]:
from pixyz.distributions import Normal, Bernoulli
from pixyz.losses import KullbackLeibler, StochasticReconstructionLoss
from pixyz.models import VAE, Model
from pixyz.utils import print_latex
x_dim = 784
z_dim = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [32]:
# Distribution APIの良さみ
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=['x'], var=['z'], name='q')
        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)
        
    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {'loc': self.fc31(h), 'scale': F.softplus(self.fc32(h))}


# generative model p(x|z)
class Generator(Bernoulli):
    def __init__(self):
        super(Generator, self).__init__(cond_var=['z'], var=['x'], name='p')
        
        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)
    
    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {'probs': torch.sigmoid(self.fc3(h))}

# distributions API
p = Generator().to(device)
q = Inference().to(device)

prior = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.),
              var=['z'], features_shape=[z_dim], name='p_{prior}').to(device)
# distributions APIに従うことでsample()だったり, 分布の掛け算などが手軽に定義できる
# 生成過程を確率分布によってモデル化するという意識を忘れずに，今自分は何を実装しているんだ？と確認させてくれる
# オレオレ実装が少なくなり得る可能性はここにあるのか？

# 確率分布として扱うからこそのloss APIの融通
kl = KullbackLeibler(q, prior)
reconst = StochasticReconstructionLoss(q, p)
loss = (kl + reconst).mean()

# これがさらに上位のmodel APIによっていい感じに定義できる
model = Model(loss=loss, distributions=[p, q],
             optimizer=optim.Adam, optimizer_params={'lr': 1e-3})

def train(epoch):
    train_loss = 0
    for x, _ in tqdm(train_loader):
        x = x.to(device)
        loss = model.train({'x': x})
        train_loss += loss
    
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

class Inference_npx(nn.Module):
    def __init__(self):
        super(Inference_npx, self).__init__()
        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)
    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {'loc': self.fc31(h), 'scale': F.softplus(self.fc32(h))}


class Generator_npx(nn.Module):
    def __init__(self):
        super(Generator_npx, self).__init__()
        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)
    
    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {'probs': torch.sigmoid(self.fc3(h))}

class VAE_npx(nn.Module):
    def __init__(self, inference, generator):
        super(VAE_npx, self).__init__()
        
        self.inference = inference
        self.generator = generator
        
    def encode(self, x):
        return self.inference(x)
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        
        # Returns a tensor with the same size as std 
        # that is filled with random numbers from a normal distribution with mean 0 and variance 1
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.generator(z)
    
    def forward(self, x):
        mu, logvar = self.encode(x)['loc'], self.encode(x)['scale']
        z = self.reparameterize(mu, logvar)
        recon_batch = self.decode(z)['probs']
        return recon_batch, mu, logvar
p = Inference_npx()
q = Generator_npx()
model_npx = VAE_npx(p, q).to(device)
optimizer = optim.Adam(params=model_npx.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    # 0.5 * sum(1 + log(sigma^2) - mu ^2 - sigma ^ 2)
    KLD = - 0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

def train_npx(epoch):
    model_npx.train()
    train_loss = 0
    for batch_idx, (data, _) in tqdm(enumerate(train_loader)):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar  = model_npx(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

In [26]:
train(3)

 12%|█▏        | 57/469 [00:04<00:29, 13.87it/s]


KeyboardInterrupt: 

In [33]:
train_npx(3)

469it [00:30, 15.14it/s]


In [ ]:
class Distribution(nn.Module):
    def __init__(self, var, cond_var=[], name='p', features_shape=torch.Size()):
        '''
        var: obj list of str
        variables of this distrbution
        cond_var conditional variables of this ditribution
        name name of this distribution
        features_shape shape of dimensions
        '''
        super().__init__()
        _vars = len(_vars) != cond_var + var
        if len(_vars) != len(set(_vars)):
            raise ValueError('there are conflicted variables')
        self._cond_var = con_var
        self._var = var
        
        self._features_shape = torch.Size(features_shape)
        self._name = convert_latex_name(name)
        
        self._prob_text = None
        self._prob_factorized_text = None
        
    @property
    def distribution_name(self):
        return ''
    
    @property
    def distribution_name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        if type(name) is str:
            self._name = name
            return
        raise ValueError('name of the distribution class must be a string type')
    
    @property
    def var(self):
        return self._var
    
    @propety
    def cond_var(self):
        return self._cond_var
    
    @property
    def input_var(self):
        # input varialbes of this distribution
        return self._cond_var
    
    @property
    def prob_text(self):
        """str: Return a formula of the (joint) probability distribution."""
        _var_text = [','.join([convert_latex_name(var_name) for var_name in self._var])]
        if len(self._cond_var) != 0:
            _var_text += [','.join([convert_latex_name(var_name) for var_name in self._cond_var])]

        _prob_text = "{}({})".format(
            self._name,
            "|".join(_var_text)
        )

        return _prob_text

    @property
    def prob_factorized_text(self):
        """str: Return a formula of the factorized probability distribution."""
        return self.prob_text

    @property
    def prob_joint_factorized_and_text(self):
        """str: Return a formula of the factorized and the (joint) probability distributions."""
        if self.prob_factorized_text == self.prob_text:
            prob_text = self.prob_text
        else:
            prob_text = "{} = {}".format(self.prob_text, self.prob_factorized_text)
        return prob_text

    @property
    def features_shape(self):
        """torch.Size or list: Shape of features of this distribution."""
        return self._features_shape

    def _check_input(self, input, var=None):
        """Check the type of given input.
        If the input type is :obj:`dict`, this method checks whether the input keys contains the :attr:`var` list.
        In case that its type is :obj:`list` or :obj:`tensor`, it returns the output formatted in :obj:`dict`.
        Parameters
        ----------
        input : :obj:`torch.Tensor`, :obj:`list`, or :obj:`dict`
            Input variables.
        var : :obj:`list` or :obj:`NoneType`, defaults to None
            Variables to check if given input contains them.
            This is set to None by default.
        Returns
        -------
        input_dict : dict
            Variables checked in this method.
        Raises
        ------
        ValueError
            Raises `ValueError` if the type of input is neither :obj:`torch.Tensor`, :obj:`list`, nor :obj:`dict.
        """
        if var is None:
            var = self.input_var

        if type(input) is torch.Tensor:
            input_dict = {var[0]: input}

        elif type(input) is list:
            # TODO: we need to check if all the elements contained in this list are torch.Tensor.
            input_dict = dict(zip(var, input))

        elif type(input) is dict:
            if not (set(list(input.keys())) >= set(var)):
                raise ValueError("Input keys are not valid.")
            input_dict = input.copy()

        else:
            raise ValueError("The type of input is not valid, got %s." % type(input))

        return input_dict
        